# Timing
## Ways to Calculate
1. contact location
- reference: https://medium.com/@hammit21/using-point-of-impact-to-measure-timing-b79ca6958221
- hit ranges:
    * home runs: 9-18 inches in front of plate
    * line drives: 3 inches in front of plate to 6 inches behind front edge of plate
    * ground balls: 6-18 inches behind the front edge of plate
- targets:
    * zone 1: 10-18 inches in front of plate
    * zone 2: 2-10 inches in front of plate
    * zone 3: 2 inches in front of plate to 6 inches behind front edge of plate
- limiting assumptions:
    * each player starts with front foot at front edge of home plate
    * ignores pitch location (depth of contact is different between inside and outside pitches)
- something to consider:
    * A batter can contact the ball in the optimal location even when their timing is off, this would be likely to cause inefficient contact. The inefficient contact could be evaluated with analysis of exit velocity and launch angle, but we would likely need more data to determine each batters peak exit velocity and optimal launch angle. (There was an article that covered some detail about determining the best exit velocity for each pitch. - https://www.mlb.com/glossary/statcast/squared-up)
- notes:
    * the ball is moving too fast to use it's location for contact location. One at bat shows the handle at 1.16 and the head at 1.67 but the closest ball location is 2.51 feet in front of the plate
    * simplest calculation of bat contact location is the average of the head and handle locations

In [1]:
import pandas as pd
import json
from pathlib import Path

pd.set_option('display.max_rows', None)

In [2]:
from build_dataframe import build_ball_flight_df, build_summary_dict, build_bat_path_df

In [70]:
with open('../data/12345634_14070.jsonl') as f:
    json_file = json.load(f)
json_file

{'units': {'length': 'foot',
  'velocity': 'mph',
  'acceleration': 'mph/s',
  'angle': 'degree'},
 'summary_acts': {'pitch': {'eventId': 'cbaf8988-2da4-43e6-affa-e0ea5670e364',
   'type': 'Sinker',
   'result': 'HitIntoPlay',
   'action': {},
   'speed': {'mph': 92.0, 'kph': 148.0, 'mps': 41.0},
   'spin': {'rpm': 2380}},
  'hit': {'eventId': 'c2f411bd-149a-4292-8743-9be188b385ba',
   'speed': {'mph': 85.0, 'kph': 136.0, 'mps': 38.0},
   'spin': {'rpm': 430}},
  'stroke': {'type': {}, 'attempt': {}}},
 'summary_score': {'runs': {'game': {'team1': 0, 'team2': 0},
   'innings': [{'team1': 0, 'team2': 0}, {'team1': 0, 'team2': 0}],
   'play': 0},
  'outs': {'inning': 1, 'play': 1},
  'count': {'balls': {'plateAppearance': 1, 'play': 0},
   'strikes': {'plateAppearance': 1, 'play': 0}}},
 'events': [{'start': {'angle': [4.649429149613672, 10.713577586436]},
   'type': 'Hit',
   'teamId': {'mlbId': 90068},
   'personId': {'mlbId': 474808052},
   'eventId': 'c2f411bd-149a-4292-8743-9be188b3

In [63]:
bat_df = build_bat_path_df(json_file)
ball_df = build_ball_flight_df(json_file)

In [64]:
hit_frame = bat_df[bat_df['event'].isin(['Hit', 'Nearest'])]
handle_loc = hit_frame['handle_pos_1'].values[0]
head_loc = hit_frame['head_pos_1'].values[0]
contact_loc = (handle_loc + head_loc) /2
contact_loc

0.0

In [65]:
quality_locations = [18/12, 10/12, 2/12, -6/12]
if contact_loc > quality_locations[0]:
    score = 0
elif contact_loc > quality_locations[1]:
    score = 4
elif contact_loc > quality_locations[2]:
    score = 3
elif contact_loc > quality_locations[3]:
    score = 2
else:
    score = 0
score

2

In [66]:
hit_frame

,event,time,head_pos_0,head_pos_1,head_pos_2,handle_pos_0,handle_pos_1,handle_pos_2
0,Hit,-2.560901,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
summary_df = pd.read_csv('summary_df.csv')
batters_df = summary_df[
    (summary_df['batter'].notnull()) 
    &
    (summary_df['action'] != 'HitByPitch')
    &
    (summary_df['pitch_result'] != 'Ball')
        # I'll want to decide what to do with this situation,
        # for now dropping it will work
        # situation = check swing?
    ].sort_values('batter')
file_list = batters_df.file_path.to_list()

In [ ]:
def process_files_in_list(bucket_name, file_list=[]):
    batter_id = 'None'
    batter_count = 0
    timing_metric_list = []
    for file_path in file_list:
        timing_metric_dict = dict()
        print(f"Processing file: {file_path}")
        json_file = read_json_from_gcs(bucket_name, file_path)
        file_batter_id = json_file['events'][0]['personId']['mlbId']
        bat_df = build_bat_path_df(json_file)
        hit_frame = bat_df[bat_df['event'].isin(['Hit', 'Nearest'])]
        handle_loc = hit_frame['handle_pos_1'].values[0]
        head_loc = hit_frame['head_pos_1'].values[0]
        contact_loc = (handle_loc + head_loc) / 2
        if batter_id = file_batter_id:
            batter_count += 1
        else:
            batter_count = 0
        timing_metric_dict['file'] = file_path
        timing_metric_dict['batter'] = file_batter_id
        timing_metric_dict['batter_count'] = batter_count
        timing_metric_dict['contact_y_loc'] = contact_loc
        timing_metric_list.append(timing_metric_dict)